# Smoking is bad

## Install and import all the necessary packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

from utils import load_smoking_data

Load the data

In [ ]:
df = load_smoking_data("smoking_data.csv")
df.head()

Chack basic structure

In [ ]:
df.shape


In [ ]:
# TODO: use .describe() on your data frame: what do you notice about the string columns?

df.describe(include='all')


Count: number of non-NaN values in the column. For example, here we can see that although we have 1314 rows, we only have 614 rows that contain the salary information.
Unique: Number of distinct value in that column
Top: The most frequent value in the column, here it is NaN
Mean: we can see that the average year of our sample is 46 years old.



In [ ]:
# TODO: do not forget to check the null values

df.isna().sum()


In [ ]:
#Drop useless columns, here just an index column
df = df.drop(columns=['Unnamed: 0'])

df.columns

In [ ]:
# TODO: prepare your data frame so that you can work with

#Convert categorical columns to numerical values
#put the function in utils.py?
df['outcome'] = df['outcome'].map({'Alive': 1, 'Dead': 0})
df['smoker'] = df['smoker'].map({'Yes': 1, 'No': 0})
df['gender'] = df['gender'].map({'male': 1, 'female': 0})


df['salary'] = df.groupby(['gender', 'smoker'])['salary'].transform(
    lambda x: x.fillna(x.median())
)

#We have 700 missing values in the 'salary' column that we will fill with the median salary of each group defined by 'gender' and 'smoker' status.
# We can not drop all rows with missing salary values as it would remove more than half of our dataset.

## Smoking analysis
Let's first check what your cousin did.

In [ ]:
# TODO: Reimplement the small code snippet described in the problem set instructions.

df.groupby("smoker").agg(prob=("outcome", np.mean))


You should get the same results. => Yes

# Is smoking really good?
Now it is up to you to run the analysis further and check what could be underlying your cousings results.

- could invistigate in confounding factors (eg. age, sex...)
- look at survival by age group
- logistic regression

In [ ]:
sns.kdeplot(data=df, x="age", hue="smoker", common_norm=False)
plt.title("Age distribution by smoking status")
plt.show()

df.groupby("smoker")["age"].describe()


We can see that smokers are young, which can inflate their suvival probaability even if smoking worsens health.

In [ ]:
bins = [0, 30, 40, 50, 60, 70, 80, 120]
labels = ["<30","30–39","40–49","50–59","60–69","70–79","80+"]
df["age_band"] = pd.cut(df["age"], bins=bins, labels=labels, right=False)

surv_by_band = (
    df.groupby(["age_band","smoker"])
      .agg(prob=("outcome","mean"), count=("outcome","size"))
      .reset_index()
)


combined = surv_by_band.copy()
combined["summary"] = combined.apply(
    lambda row: f"{row['prob']:.3f}  (n={row['count']})",
    axis=1
)

nice_table = combined.pivot(index="age_band", columns="smoker", values="summary")
nice_table.columns = ["Non-smoker", "Smoker"]

nice_table




Smokers are much younger on average.

The “<30” and “30–39” categories have many smokers

The older categories have far fewer smokers

So the smoker group is “loaded” with young people — who naturally have extremely high survival regardless of smoking.

This creates the illusion that smokers have higher survival.

This is Simpson’s paradox:

A trend appears in aggregate data
but reverses when you disaggregate by a confounding variable (here: age).